In [291]:
import numpy as np
import matplotlib.pyplot as plt
%config InlineBackend.figure_format='svg'

# Basis functions

In [526]:
def triangle_basis(coords,x,y):
    # This code currently works, although the way coordinates are passed may need to change
    names = ['x1','y1','x2','y2','x3','y3']
    d = {}
    i = 0
    for coord in coords:
        d[names[i]]=coord[0]
        d[names[i+1]]=coord[1]
        i += 2
    arr = np.array([[1,d['x1'],d['y1']],
                    [1,d['x2'],d['y2']],
                    [1,d['x3'],d['y3']]])
    area = 0.5*np.linalg.det(arr)
    X,Y = np.meshgrid(x,y)
    l1 = ((d['x2']*d['y3']-d['x3']*d['y2']) + (d['y2']-d['y3'])*X + (d['x3']-d['x2'])*Y)/(2*area)
    l2 = ((d['x3']*d['y1']-d['x1']*d['y3']) + (d['y3']-d['y1'])*X + (d['x1']-d['x3'])*Y)/(2*area)
    l3 = ((d['x1']*d['y2']-d['x2']*d['y1']) + (d['y1']-d['y2'])*X + (d['x2']-d['x1'])*Y)/(2*area)
    return l1,l2,l3

def square_basis(coords):
    x = coords[:len(coords)//2]
    y = coords[len(coords)//2:]

    x_ = np.linspace(x.min(),x.max())
    y_ = np.linspace(y.min(),y.max())
    X,Y = np.meshgrid(x_,y_)
    
    N1 = (X-x[1])/(x[0]-x[1])*(Y-y[3])/(y[0]-y[3])
    N2 = (X-x[0])/(x[1]-x[0])*(Y-y[2])/(y[1]-y[2])
    N3 = (X-x[3])/(x[2]-x[3])*(Y-y[1])/(y[2]-y[1])
    N4 = (X-x[2])/(x[3]-x[2])*(Y-y[0])/(y[3]-y[0])
    return [N1,N2,N3,N4]

def sq_basis_derivs(coords):
    x = coords[:len(coords)//2]
    y = coords[len(coords)//2:]
    a = (x.max()-x.min())/2
    b = (y.max()-y.min())/2
    x_ = np.linspace(x.max(),x.min())
    y_ = np.linspace(y.max(),y.min())
    X,Y = np.meshgrid(x_,y_)
    dN1dx = 1/(x[0]-x[1])*(Y-y[3])/(y[0]-y[3])
    dN1dy = (X-x[1])/(x[0]-x[1])*1/(y[0]-y[3])
    dN2dx = 1/(x[1]-x[0])*(Y-y[2])/(y[1]-y[2])
    dN2dy = (X-x[0])/(x[1]-x[0])*1/(y[1]-y[2])
    dN3dx = 1/(x[2]-x[3])*(Y-y[1])/(y[2]-y[1])
    dN3dy = (X-x[3])/(x[2]-x[3])*1/(y[2]-y[1])
    dN4dx = 1/(x[3]-x[2])*(Y-y[0])/(y[3]-y[0])
    dN4dy = (X-x[2])/(x[3]-x[2])*1/(y[3]-y[0])
    return [dN1dx,dN2dx,dN3dx,dN4dx], [dN1dy,dN2dy,dN3dy,dN4dy]

def ref_triangle_basis():
    xi = np.linspace(0,1)
    eta = np.linspace(0,1)
    Xi, Eta = np.meshgrid(xi,eta)
    N1 =1-Eta-Xi
    N2 = Xi
    N3 = Eta
    return N1,N2,N3

def ref_square_basis():
    xi = np.linspace(-1,1)
    eta = np.linspace(-1,1)
    Xi, Eta = np.meshgrid(xi,eta)
    N1 = 0.25*(1+Xi)*(1+Eta)
    N2 = 0.25*(1-Xi)*(1+Eta)
    N3 = 0.25*(1-Xi)*(1-Eta)
    N4 = 0.25*(1+Xi)*(1-Eta)
    return N1,N2,N3,N4

def square_basis_derivs():
    xi = np.linspace(-1,1,5)
    eta = np.linspace(-1,1,5)
    Xi, Eta = np.meshgrid(xi,eta)
    dN1xi = 0.25*(1+Eta)
    dN2xi = -0.25*(1+Eta)
    dN3xi = -0.25*(1-Eta)
    dN4xi = 0.25*(1-Eta)
    dN1eta = 0.25*(1+Xi)
    dN2eta = 0.25*(1-Xi)
    dN3eta = -0.25*(1-Xi)
    dN4eta = -0.25*(1+Xi)
    return [dN1xi,dN2xi,dN3xi,dN4xi],[dN1eta,dN2eta,dN3eta,dN4eta]

def square_jacobian(coords,dNxi,dNeta):
    # coordinates in this function are passed x first, then y
    x = np.array(coords[:len(coords)//2])
    y = np.array(coords[len(coords)//2:])
    dxdxi = sum([x_*dN for x_,dN in zip(x,dNxi)])
    dxdeta = sum([x_*dN for x_,dN in zip(x,dNeta)])
    dydxi = sum([y_*dN for y_,dN in zip(y,dNxi)])
    dydeta = sum([y_*dN for y_,dN in zip(y,dNeta)])
    
    return np.block([[dxdxi,dxdeta],[dydxi,dydeta]])

In [527]:
coords = [(0.,0.),(0.,1.),(1.,0.)]
x = np.linspace(0,1)
y = np.linspace(0,1)
N1,N2,N3 = triangle_basis(coords,x,y)

In [294]:
%matplotlib qt
fig,ax = plt.subplots(subplot_kw={'projection':'3d'})
x = np.linspace(-1,1)
y = np.linspace(-1,1)
X,Y = np.meshgrid(x,y)
N1,N2,N3,N4 = ref_square_basis()
# my_map = Y<=1-X # show only the triangular region
surf = ax.plot_surface(X,Y,N1)
ax.set_xlabel('X')
ax.set_ylabel('Y')
plt.show()

In [295]:
dNxi,dNeta = square_basis_derivs()

In [296]:
# coords are counterclockwise from upper right corner -- (1,1) in the reference element
coords = [1.,0.,0.,1.,2.,2.,1.,1.]
dxdxi = square_jacobian(coords,dNxi,dNeta)

In [297]:
dxdxi.shape

(10, 10)

# Meshing Functions

In [528]:
def mesher2D(nrows,ncols,length,height,deg=1):
    '''
    This is a function for creating a mesh of 2D rectilinear elements. It assumes the points are laid out in a grid.
    Currently, only linear elements are supported.

    Inputs
    ------
    nrows: integer. Number of elements in a row
    ncols: integer. Number of elements in a column
    length: float. Length of the domain
    height: float. Height of the domain

    Returns
    -------
    node_dict: dictionary. A multi-level dictionary with the global node numbers as the outer keys, and the inner dictionary
    for each global node containing the x and y coordinates of that node.
    
    elem_nodes: array. An array where each row is an element, counting left to right, bottom to top of the domain,
    and each entry for each element are the global node numbers, counting counter-clockwise from the upper right node.

    '''
    if nrows*ncols%2 != 0:
        root = np.sqrt(nrows*ncols)
        if root - int(root) > 0.0: # checks for a perfect square
            raise ValueError('Uneven grid detected.')
        # else:
        #     print('Square domain detected.')
    n_xnodes = ncols + 1
    n_ynodes = nrows + 1
    dx = length/ncols
    dy = height/nrows
    xnodes = np.arange(0,length+dx,dx)
    ynodes = np.arange(0,height+dy,dy)
    X,Y = np.meshgrid(xnodes,ynodes)
    if deg==1:
        nodes_per_elem=4
    elem_nodes = np.zeros((nrows*ncols,nodes_per_elem),dtype='int8')
    # row = 0
    # for e in range(nrows*ncols):
    #     # print(e+row,e+1+row,e+nrows+1+row,e+nrows+2+row)
    #     elem_nodes[e] = e+nrows+2+row,e+nrows+1+row,e+row,e+1+row
    #     if (e%(ncols-1) == 0.0 and e!=0):
    #         row += 1
    for row in range(nrows):
        for col in range(ncols):
            elem_nodes[nrows*row+col] = n_xnodes*(row+1)+(col+1),n_xnodes*(row+1)+col,n_xnodes*row+col,n_xnodes*row+col+1
    node_dict = {}
    node_count = 0
    for i in range(nrows+1):
        for j in range(ncols+1):
            node_dict[node_count] = {'x':X[i,j],'y':Y[i,j]}
            node_count += 1
    return node_dict, elem_nodes

In [299]:
node_dict, elem_nodes = mesher2D(4,4,2,2)

In [300]:
elem_nodes

array([[ 6,  5,  0,  1],
       [ 7,  6,  1,  2],
       [ 8,  7,  2,  3],
       [ 9,  8,  3,  4],
       [11, 10,  5,  6],
       [12, 11,  6,  7],
       [13, 12,  7,  8],
       [14, 13,  8,  9],
       [16, 15, 10, 11],
       [17, 16, 11, 12],
       [18, 17, 12, 13],
       [19, 18, 13, 14],
       [21, 20, 15, 16],
       [22, 21, 16, 17],
       [23, 22, 17, 18],
       [24, 23, 18, 19]], dtype=int8)

# Gathering Coordinates
Create a helper function that returns the coordinates of the nodes of an elements in such a way that it can be used by other functions.

In [404]:
def retrieve_coords(node_dict,elem_nodes,eid):
    nodes = elem_nodes[eid]
    x = []
    y = []
    for node in nodes:
        xval = node_dict[node]['x']
        yval = node_dict[node]['y']
        x.append(xval)
        y.append(yval)
    coords = np.concatenate((x,y))
    return coords

In [405]:
coords = retrieve_coords(node_dict,elem_nodes,1)
coords

array([0.26666667, 0.13333333, 0.13333333, 0.26666667, 0.13333333,
       0.13333333, 0.        , 0.        ])

# Creating the Coefficient Matrices

In [406]:
def cond_mat(dNdxi,dNdeta,k,J=None):
    # J defaults to none for non-isoparametric reference elements
    K1 = np.concatenate((dNdxi[0],dNdeta[0]),axis=1)
    K2 = np.concatenate((dNdxi[0],dNdeta[0]),axis=0)
    if J is not None:
        Jinv = np.linalg.inv(J)
    # dNdxi_col = np.array(dNdxi).reshape(-1,1)
    # dNdxi_row = np.array(dNdxi).reshape(1,-1)
    # dNdeta_col = np.array(dNdeta).reshape(-1,1)
    # dNdeta_row = np.array(dNdeta).reshape(1,-1)
    for dNx, dNe in zip(dNdxi[1:],dNdeta[1:]):
        k1 = np.concatenate((dNx,dNe),axis=1)
        k2 = np.concatenate((dNx,dNe),axis=0)
        K1 = np.concatenate((K1,k1),axis=0)
        K2 = np.concatenate((K2,k2),axis=1)
    if J==None:
        Kint = k*K1@K2
    else:
        Kint = k*np.linalg.det(J)*(K1@Jinv.T@Jinv@K1.T)
    return Kint
    
def conv_mat(N,hz,lin=True):
    if lin:
        n = 4
    rows, cols = N[0].shape
    Kint = np.zeros((n*rows,n*cols))
    for row in range(n):
        for col in range(n):
            Kint[row*rows:row*rows+rows,col*cols:col*cols+cols] = hz*N[row]*N[col]
    return Kint

def rhs_vec(N,hz,k,Tinf,coords,bcs_arr,elem_nodes,eid,qtop,qbot,qright):
    '''
    Linear rectilinear elements only currently.
    Boundary conditions will also be treated here.
    '''
    nodes = elem_nodes[eid]
    x = coords[:len(coords)//2]
    y = coords[len(coords)//2:]
    rows,cols = N[0].shape
    x1 = x.min()
    x2 = x.max()
    y1 = y.min()
    y2 = y.max()
    x_ = np.linspace(x1,x2,rows)
    y_ = np.linspace(y1,y2,cols)
    F = np.zeros(4)

    ### BOUNDARY CONDITIONS ###
    # 1D shape functions for treating boundary conditions
    N1x = (x_-x1)/(x2-x1)
    N2x = (x_-x2)/(x1-x2)
    N1y = (y_-y1)/(y2-y1)
    N2y = (y_-y2)/(y1-y2)
    for i in range(4):
        Z = hz*Tinf*N[i]
        xint = np.trapz(Z,x=x_,axis=1)
        integrand = np.trapz(xint,x=y_)
        F[i] += integrand

    bcs_arr = bcs_arr.flatten()
    # Case 1: Top boundary
    if (bcs_arr[nodes[0]] and bcs_arr[nodes[1]]) >= 2:
        q = qtop*N1x
        xint = np.trapz(q,x=x_)
        F[0] += xint
        q = qtop*N2x
        xint = np.trapz(q,x=x_)
        F[1] += xint
    # Case 2: Bottom boundary
    if (bcs_arr[nodes[2]] and bcs_arr[nodes[3]]) >= 2:
        q = qbot*N2x
        xint = np.trapz(q,x=x_)
        F[2] += xint
        q = qbot*N1x
        xint = np.trapz(q,x=x_)
        F[3] += xint    
    # Case 3: Right boundary
    if (bcs_arr[nodes[3]] and bcs_arr[nodes[0]]) >= 2:
        q = qright*N2y
        yint = np.trapz(q,x=y_)
        F[3] += yint
        q = qright*N1y
        yint = np.trapz(q,x=y_)
        F[0] += yint
    return F
    

In [407]:
dNx,dNy = sq_basis_derivs(coords)
N1,N2,N3,N4 = square_basis(coords)
x = coords[:len(coords)//2]
y = coords[:len(coords)//2]
x_ = np.linspace(x.min(),x.max())
y_ = np.linspace(y.min(),y.max())
X,Y = np.meshgrid(x_,y_)
fig,ax = plt.subplots(subplot_kw={'projection':'3d'})
# my_map = Y<=1-X # show only the triangular region
surf = ax.plot_surface(X,Y,N1+N2)
ax.set_xlabel('X')
ax.set_ylabel('Y')
plt.show()

# Double integration algorithm that returns the 4x4 matrix for linear rectilinear elements

In [408]:
def integrateK(K,coords):
    '''
    Linear rectilinear elements only currently
    '''
    x = coords[:len(coords)//2]
    y = coords[len(coords)//2:]
    rows,cols = K.shape
    nx = rows//4
    x1 = x.min()
    x2 = x.max()
    y1 = y.min()
    y2 = y.max()
    x_ = np.linspace(x1,x2,nx)
    y_ = np.linspace(y1,y2,nx)
    Kint = np.zeros((4,4))
    for i in range(4):
        for j in range(4):
            xint = np.trapz(K[i*nx:i*nx+nx,j*nx:j*nx+nx],x=x_,axis=1)
            integrand = np.trapz(xint,x=y_)
            Kint[i,j] = integrand
    return Kint
    

# Boundary conditions
For this particular problem, there are essential boundary conditions on the left, and natural boundary conditions everywhere else

In [409]:
def create_bcs(nrows,ncols):
    nnodes = 2*nrows*ncols+1
    nx = nrows + 1
    ny = ncols + 1
    # x = coords[:len(coords)//2]
    # y = coords[len(coords)//2:]
    # nnx,nny = N[0].shape
    # x1 = x.min()
    # x2 = x.max()
    # y1 = y.min()
    # y2 = y.max()
    # x_ = np.linspace(x1,x2,nnx)
    # y_ = np.linspace(y1,y2,nny)
    mesh = np.zeros((ny,nx))
    mesh[:,0] = 1
    mesh[0,1:] = 2
    mesh[-1,1:] = 2
    mesh[:,-1] = 2
    mesh[0,0] = 3
    mesh[-1,0] = 3
    # f = np.zeros(4)
    # nodes = elem_nodes[eid]
    # for i in range(3):
    #     if mesh[nodes[i]] and mesh[nodes[i+1]] > 1.:
    #         if i == 0:
    #             q = (N[0][0,:]+N[1][0,:])*qtop
    #             integrand = np.trapz()


    # vals = np.zeros((ny,nx))
    # vals[:,0] = Twall
    # vals[0,1:] = qbot
    # vals[-1,1:] = qtop
    # vals[:,-1] = qright
    # vals[0,-1] = qbot + qright
    # vals[-1,-1] = qtop + qright
    return mesh

In [410]:
print(elem_nodes)
print(coords)

[[  17   16    0    1]
 [  18   17    1    2]
 [  19   18    2    3]
 [  20   19    3    4]
 [  21   20    4    5]
 [  22   21    5    6]
 [  23   22    6    7]
 [  24   23    7    8]
 [  25   24    8    9]
 [  26   25    9   10]
 [  27   26   10   11]
 [  28   27   11   12]
 [  29   28   12   13]
 [  30   29   13   14]
 [  31   30   14   15]
 [  33   32   16   17]
 [  34   33   17   18]
 [  35   34   18   19]
 [  36   35   19   20]
 [  37   36   20   21]
 [  38   37   21   22]
 [  39   38   22   23]
 [  40   39   23   24]
 [  41   40   24   25]
 [  42   41   25   26]
 [  43   42   26   27]
 [  44   43   27   28]
 [  45   44   28   29]
 [  46   45   29   30]
 [  47   46   30   31]
 [  49   48   32   33]
 [  50   49   33   34]
 [  51   50   34   35]
 [  52   51   35   36]
 [  53   52   36   37]
 [  54   53   37   38]
 [  55   54   38   39]
 [  56   55   39   40]
 [  57   56   40   41]
 [  58   57   41   42]
 [  59   58   42   43]
 [  60   59   43   44]
 [  61   60   44   45]
 [  62   61

# Program Flow
Attempting to make the program flow together, in addition to assembling the global matrices

In [529]:
def globalK(elem_nodes,eid,k,K):
    nodes = elem_nodes[eid]
    rows,cols = k.shape
    nodes_rows = int(np.sqrt(len(nodes)))
    nodes.reshape(nodes_rows,nodes_rows)
    for row in range(rows):
        for col in range(cols):
            K[nodes[row],nodes[col]] += k[row,col]
    return K

def globalF(elem_nodes,eid,f,F):
    nodes = elem_nodes[eid]
    rows = len(f)
    for row in range(rows):
        F[nodes[row]] += f[row]
    return F
            
nelemsx = 2
nelemsy = 2
nnodes = (nelemsx+1)*(nelemsy+1)
L = 2.
H = 2.
x = np.linspace(0,L,nelemsx+1)
y = np.linspace(0,H,nelemsy+1)
X,Y = np.meshgrid(x,y)
t = 0.05
Twall = 80.
Tinf = 30.
kcond = 5.
hconv = 10.
qtop = -500.
qright = -1000.
qbot = -1500.
hz = 2*hconv/t
node_dict, elem_nodes = mesher2D(nelemsy,nelemsx,L,H,1)
K = np.zeros((nnodes,nnodes))
F = np.zeros(nnodes)

In [530]:
for eid in range(nelemsx*nelemsy):
    coords = retrieve_coords(node_dict,elem_nodes,eid)
    bcs_arr = create_bcs(nelemsy,nelemsx)
    N = square_basis(coords)
    dNx,dNy = sq_basis_derivs(coords)
    k1 = cond_mat(dNx,dNy,kcond,None)
    k2 = conv_mat(N,hz,True)
    kint = integrateK(k1+k2,coords)
    K = globalK(elem_nodes,eid,kint,K)
    print(K)
    print('*'*20)
    fint = rhs_vec(N,hz,kcond,Tinf,coords,bcs_arr,elem_nodes,eid,qtop,qbot,qright)
    F = globalF(elem_nodes,eid,fint,F)
    print(F)
for idx,bcs in enumerate(bcs_arr.flatten()):
    if (bcs == 1 or bcs == 3):
        K[idx,idx] = 1e10
        F[idx] = Twall*1e10

    

[4 3 0 1]
4
3
0
1
[[ 169.46295718   22.21759259    0.           22.21759259 -113.89814236
     0.            0.            0.            0.        ]
 [  22.21759259  169.46295718    0.         -113.89814236   22.21759259
     0.            0.            0.            0.        ]
 [   0.            0.            0.            0.            0.
     0.            0.            0.            0.        ]
 [  22.21759259 -113.89814236    0.          169.46295718   22.21759259
     0.            0.            0.            0.        ]
 [-113.89814236   22.21759259    0.           22.21759259  169.46295718
     0.            0.            0.            0.        ]
 [   0.            0.            0.            0.            0.
     0.            0.            0.            0.        ]
 [   0.            0.            0.            0.            0.
     0.            0.            0.            0.        ]
 [   0.            0.            0.            0.            0.
     0.            0.    

In [424]:
T = np.linalg.solve(K,F)
print(T)

[80.00000083 66.68958393 57.36987017 49.93447337 44.46510593 40.09130409
 36.89689677 34.33226015 32.52696289 31.08056481 30.16702221 80.00000165
 67.85772571 58.18937082 51.0736269  45.39877067 41.21534915 37.86499523
 35.44828423 33.50767834 32.19305589 31.15108772 80.00000167 68.12078152
 58.94648278 51.6692836  46.23289571 41.89795381 38.71276955 36.1603775
 34.35740438 32.91515801 32.00086597 80.00000167 68.37119195 59.22902715
 52.22204564 46.6691791  42.51582014 39.20272119 36.79475908 34.86704008
 33.55354907 32.51563581 80.00000168 68.45966159 59.5004785  52.45134681
 47.0790229  42.81898774 39.65382877 37.12807568 35.33042044 33.89785631
 32.98171737 80.00000168 68.54742942 59.58682671 52.6753858  47.22822529
 43.10890199 39.83794053 37.44124377 35.53034994 34.21830664 33.18613273
 80.00000168 68.5575607  59.67535045 52.70862611 47.37766124 43.165967
 40.01780014 37.51489269 35.72231521 34.29941559 33.38122014 80.00000168
 68.56991803 59.6329425  52.74612136 47.32698016 43.22

In [425]:
i = int(np.sqrt(len(T)))
plt.imshow(T.reshape(i,i),origin='lower')

In [426]:
plt.contourf(X,Y,T.reshape(i,i),vmin=30,vmax=80,levels=20)
plt.grid()
plt.colorbar()

# Same thing, but with Gaussian Quadrature
In order to use the reference elements, I will now implement the Gaussian quadrature. Some of the functions are repeated below for ease of debugging.

In [537]:
def mesher2D(nrows,ncols,length,height,deg=1):
    '''
    This is a function for creating a mesh of 2D rectilinear elements. It assumes the points are laid out in a grid.
    Currently, only linear elements are supported.

    Inputs
    ------
    nrows: integer. Number of elements in a row
    ncols: integer. Number of elements in a column
    length: float. Length of the domain
    height: float. Height of the domain

    Returns
    -------
    node_dict: dictionary. A multi-level dictionary with the global node numbers as the outer keys, and the inner dictionary
    for each global node containing the x and y coordinates of that node.
    
    elem_nodes: array. An array where each row is an element, counting left to right, bottom to top of the domain,
    and each entry for each element are the global node numbers, counting counter-clockwise from the upper right node.

    '''
    if nrows*ncols%2 != 0:
        root = np.sqrt(nrows*ncols)
        if root - int(root) > 0.0: # checks for a perfect square
            raise ValueError('Uneven grid detected.')
        # else:
        #     print('Square domain detected.')
    n_xnodes = ncols + 1
    n_ynodes = nrows + 1
    dx = length/ncols
    dy = height/nrows
    xnodes = np.arange(0,length+dx,dx)
    ynodes = np.arange(0,height+dy,dy)
    X,Y = np.meshgrid(xnodes,ynodes)
    if deg==1:
        nodes_per_elem=4
    elem_nodes = np.zeros((nrows*ncols,nodes_per_elem),dtype='int8')
    # row = 0
    # for e in range(nrows*ncols):
    #     # print(e+row,e+1+row,e+nrows+1+row,e+nrows+2+row)
    #     elem_nodes[e] = e+nrows+2+row,e+nrows+1+row,e+row,e+1+row
    #     if (e%(ncols-1) == 0.0 and e!=0):
    #         row += 1
    for row in range(nrows):
        for col in range(ncols):
            elem_nodes[nrows*row+col] = n_xnodes*(row+1)+(col+1),n_xnodes*(row+1)+col,n_xnodes*row+col,n_xnodes*row+col+1
    node_dict = {}
    node_count = 0
    for i in range(nrows+1):
        for j in range(ncols+1):
            node_dict[node_count] = {'x':X[i,j],'y':Y[i,j]}
            node_count += 1
    return node_dict, elem_nodes

def retrieve_coords(node_dict,elem_nodes,eid):
    nodes = elem_nodes[eid]
    print(nodes)
    x = []
    y = []
    for node in nodes:
        print(node)
        xval = node_dict[node]['x']
        yval = node_dict[node]['y']
        x.append(xval)
        y.append(yval)
    coords = np.concatenate((x,y))
    return coords

def ref_square_basis(Xi,Eta):
    N1 = 0.25*(1+Xi)*(1+Eta)
    N2 = 0.25*(1-Xi)*(1+Eta)
    N3 = 0.25*(1-Xi)*(1-Eta)
    N4 = 0.25*(1+Xi)*(1-Eta)
    return [N1,N2,N3,N4]

def square_basis_derivs(Xi,Eta):
    dN1xi = 0.25*(1+Eta)
    dN2xi = -0.25*(1+Eta)
    dN3xi = -0.25*(1-Eta)
    dN4xi = 0.25*(1-Eta)
    dN1eta = 0.25*(1+Xi)
    dN2eta = 0.25*(1-Xi)
    dN3eta = -0.25*(1-Xi)
    dN4eta = -0.25*(1+Xi)
    return [dN1xi,dN2xi,dN3xi,dN4xi],[dN1eta,dN2eta,dN3eta,dN4eta]

def square_jacobian(coords,dNxi,dNeta):
    dNxi=np.array(dNxi)
    dNeta=np.array(dNeta)
    # coordinates in this function are passed x first, then y
    x = np.array(coords[:len(coords)//2])
    y = np.array(coords[len(coords)//2:])
    dxdxi = np.dot(x,dNxi)
    dxdeta = np.dot(x,dNeta)
    dydxi = np.dot(y,dNxi)
    dydeta = np.dot(y,dNeta)
    # dxdxi = sum([x_*dN for x_,dN in zip(x,dNxi)])
    # dxdeta = sum([x_*dN for x_,dN in zip(x,dNeta)])
    # dydxi = sum([y_*dN for y_,dN in zip(y,dNxi)])
    # dydeta = sum([y_*dN for y_,dN in zip(y,dNeta)])
    return np.array([[dxdxi,dydxi],[dxdeta,dydeta]])

def gauss_points():
    '''
    Gaussian quadrature weights and points in the reference element
    '''
    # 4-point square, CCW from upper right
    xi1 = np.sqrt(1/3)
    xi2 = -np.sqrt(1/3)
    xi3 = -np.sqrt(1/3)
    xi4 = np.sqrt(1/3)
    eta1 = np.sqrt(1/3)
    eta2 = np.sqrt(1/3)
    eta3 = -np.sqrt(1/3)
    eta4 = -np.sqrt(1/3)
    weights = np.array([1.,1.,1.,1.])
    xi = np.array([xi1,xi2,xi3,xi4])
    eta = np.array([eta1,eta2,eta3,eta4])
    return xi, eta, weights

def globalK(elem_nodes,eid,k,K):
    nodes = elem_nodes[eid]
    rows,cols = k.shape
    nodes_rows = int(np.sqrt(len(nodes)))
    nodes.reshape(nodes_rows,nodes_rows)
    for row in range(rows):
        for col in range(cols):
            K[nodes[row],nodes[col]] += k[row,col]
    return K

def globalF(elem_nodes,eid,f,F):
    nodes = elem_nodes[eid]
    rows = len(f)
    for row in range(rows):
        F[nodes[row]] += f[row]
    return F
            
def KCond(kcond,dNxi,dNeta,J):
    Jinv = np.linalg.inv(J)
    dNx = np.array(dNxi).reshape(-1,1)
    dNe = np.array(dNeta).reshape(-1,1)
    K1 = np.hstack((dNx,dNe))
    K2 = np.vstack((dNx.T,dNe.T))
    K = kcond*K1@Jinv.T@Jinv@K2*np.linalg.det(J)
    return K

def KConv(hz,N,J):
    Narr = np.array(N).reshape(-1,1)
    K = hz*np.outer(Narr,Narr.T)*np.linalg.det(J)
    return K

def fluxes(nelemsx,nelemsy,nodes,qtop,qbot,qright):
    nxnodes = nelemsx + 1
    nynodes = nelemsy + 1
    nnodes = nxnodes*nynodes
    arr = np.zeros((nynodes,nxnodes))
    arr[:,0] += 1
    arr[0,:] += 2
    arr[-1,:] += 2
    arr[:,-1] += 2
    arr = arr.flatten()
    fluxes = {}
    if arr[nodes[0]] >= 2 and arr[nodes[1]] >= 2:
        fluxes['top'] = qtop
    else:
        fluxes['top'] = 0.0
    if arr[nodes[2]] >= 2 and arr[nodes[3]] >= 2:
        fluxes['bottom'] = qbot
    else:
        fluxes['bottom'] = 0.0
    if arr[nodes[3]] >= 2 and arr[nodes[0]] >= 2:
        fluxes['right'] = qright
    else:
        fluxes['right'] = 0.0
    fluxes['left'] = 0.0
    return fluxes, arr

def f_bcs(coords,fluxes,edge):
    x = coords[:len(coords)//2]
    y = coords[len(coords)//2:]
    f = np.zeros(4)
    W = [1.0,1.0]
    q = fluxes[edge]
    if edge == 'top':
        Etav = [1.0, 1.0]
        Eta = [np.sqrt(1/3),np.sqrt(1/3)]
        Xiv = [np.sqrt(1/3),-np.sqrt(1/3)]
        Xi = [np.sqrt(1/3),-np.sqrt(1/3)]
        idx = [0,1]
    if edge == 'left':
        Xiv = [-1.0, -1.0]
        Xi = [-np.sqrt(1/3),-np.sqrt(1/3)]
        Eta = [np.sqrt(1/3),-np.sqrt(1/3)]
        Etav = [np.sqrt(1/3),-np.sqrt(1/3)]
        idx = [1,2]
    if edge == 'bottom':
        Etav = [-1.0, -1.0]
        Eta = [-np.sqrt(1/3),-np.sqrt(1/3)]
        Xiv = [-np.sqrt(1/3),np.sqrt(1/3)]
        Xi = [-np.sqrt(1/3),np.sqrt(1/3)]
        idx = [2,3]
    if edge == 'right':
        Xiv = [1.0, 1.0]
        Xi = [np.sqrt(1/3),np.sqrt(1/3)]
        Etav = [-np.sqrt(1/3),np.sqrt(1/3)]
        Eta = [-np.sqrt(1/3),np.sqrt(1/3)]
        idx = [3,0]
    for i,xiv,etav,xi,eta,w in zip(idx,Xiv,Etav,Xi,Eta,W):
        Nv = np.array(ref_square_basis(xiv,etav))

        N = ref_square_basis(xi,eta)
        dNx,dNe = square_basis_derivs(xi,eta)
        dTdn = (Nv[idx[0]]+Nv[idx[1]])*q
        if edge == 'top' or edge == 'bottom':
            dN = dNx
        else:
            dN = dNe
        transform = sum(dN*x)**2 + sum(dN*y)**2
        f += Nv*dTdn*np.sqrt(transform)*w

        
    return f

def f_rhs(N,Tinf,hz,J):
    N = np.array(N)
    return N*hz*Tinf*np.linalg.det(J)
    

In [538]:
nelemsx = 2
nelemsy = 2
nnodes = (nelemsx+1)*(nelemsy+1)
L = 2.
H = 2.
x = np.linspace(0,L,nelemsx+1)
y = np.linspace(0,H,nelemsy+1)
X,Y = np.meshgrid(x,y)
t = 0.05
Twall = 80.
Tinf = 30.
kcond = 5.
hconv = 10.
qtop = -500
qright = -1000.
qbot = -1500.
hz = 2*hconv/t
node_dict, elem_nodes = mesher2D(nelemsy,nelemsx,L,H,1)

In [539]:
# testing with example from book
coords = [2,0,0,1,2,1,0,0]
xi, eta, w = gauss_points()
for x,e,w in zip(xi,eta,w):
    dNxi, dNeta = square_basis_derivs(x,e)
    J = square_jacobian(coords,dNxi,dNeta)
    print(J)

[[0.89433757 0.39433757]
 [0.39433757 0.89433757]]
[[0.89433757 0.39433757]
 [0.10566243 0.60566243]]
[[0.60566243 0.10566243]
 [0.10566243 0.60566243]]
[[0.60566243 0.10566243]
 [0.39433757 0.89433757]]


In [540]:
K = np.zeros((nnodes,nnodes))
F = np.zeros(nnodes)
for e in range(nelemsx*nelemsy):
    print(F)
    coords = retrieve_coords(node_dict,elem_nodes,e)
    nodes = elem_nodes[e]
    muh_fluxes, bcs_arr = fluxes(nelemsx,nelemsy,nodes,qtop,qbot,qright)
    x = coords[:len(coords)//2]
    y = coords[len(coords)//2:]
    Xi,Eta,W = gauss_points()
    # dNx,dNe = square_basis_derivs(Xi,Eta)
    Kcond = np.zeros((4,4))
    Kconv = np.zeros((4,4))
    Fbcs = np.zeros(4)
    Frhs = np.zeros(4)
    edges = ['top','left','bottom','right']
    for xi,eta,w,edge in zip(Xi,Eta,W,edges):
        N = ref_square_basis(xi,eta)
        dNx,dNe = square_basis_derivs(xi,eta)
        J = square_jacobian(coords,dNx,dNe)
        ktemp = KCond(kcond,dNx,dNe,J)
        Kcond += ktemp*w
        ktemp = KConv(hz,N,J)
        Kconv += ktemp*w
        fbcs = f_bcs(coords,muh_fluxes,edge)
        Fbcs += fbcs*w
        frhs = f_rhs(N,Tinf,hz,J)
        Frhs += frhs*w
    for i,row in enumerate(nodes):
        F[row] += (Fbcs[i]+Frhs[i])
        for j,col in enumerate(nodes):
            K[row,col] += (Kcond[i,j]+Kconv[i,j]) 
    
for idx,node in enumerate(bcs_arr):
    if node == 1 or node == 3:
        K[idx,idx] = 10e8
        F[idx] = Twall*10e8   

[0. 0. 0. 0. 0. 0. 0. 0. 0.]
[4 3 0 1]
4
3
0
1
[2250. 2250.    0. 3000. 3000.    0.    0.    0.    0.]
[5 4 1 2]
5
4
1
2
[2250. 4500. 1750. 3000. 6000. 2500.    0.    0.    0.]
[7 6 3 4]
7
6
3
4
[2250. 4500. 1750. 6000. 9000. 2500. 2750. 2750.    0.]
[8 7 4 5]
8
7
4
5


In [524]:
sol = np.linalg.solve(K,F)

In [525]:
i = int(np.sqrt(len(sol)))
x = np.linspace(0,L,nelemsx+1)
y = np.linspace(0,H,nelemsy+1)
X,Y = np.meshgrid(x,y)
Z = sol.reshape(i,i)
plt.contourf(X,Y,Z)
plt.colorbar()
plt.show()


In [544]:
# SOMETHING IS WRONG WITH THE MESHER FOR HIGHER ELEMENT COUNTS


64